In [13]:
import myosuite
import gym
import skvideo.io
import numpy as np
import os
import imageio
import tqdm
import time

In [14]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 400):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")


<h2> Create an A matrix with T x N (time x mean firing rate)

Get Policy 

In [16]:
policy = "scripts/iterations/best_policy.pickle" #Change the folder to the policy location

import pickle
# load policy
pi = pickle.load(open(policy, 'rb'))



Setup data collection

In [17]:
# Use sleep if you want to examine certain steps during the episodes
from time import sleep


env = gym.make('CenterReachOut-v0')
frames = []

NeuralData = {}
MuscleActData = {}
FiberJointData = {}

for ep_num in range(108):  #Number of episodes
  NeuralData[ep_num] = []
  MuscleActData[ep_num] = []
  FiberJointData[ep_num] = []
  
  timestep = 0
  env.reset()
  print(ep_num)
  ep_rewards = []
  done = False
  obs = env.reset()
  for ac_num in range(16): #Number of steps per episode / Each time step is 0.05 seconds
      timestep += 0.05
      frame = env.sim.renderer.render_offscreen(
                        width=400,
                        height=400,
                        camera_id=1)
      frames.append(frame)
      
      # We're appending data of each timestep to an epsiode number key
            
      NeuralData[ep_num].append(np.concatenate((pi.show_activations()['layer_0'][0], pi.show_activations()['layer_1'][0])))
      MuscleActData[ep_num].append(env.obs_dict['act'])
      FiberJointData[ep_num].append(np.concatenate((env.obs_dict['hand_qpos'],env.obs_dict['hand_qvel'], env.sim.data.actuator_length[:].copy(), env.sim.data.actuator_velocity[:].copy())))


      o = env.get_obs()
      action, _ = pi.get_action(o)
      obs, reward, done, info = env.step(action)
      
      

env.close()


os.makedirs('videos', exist_ok=True)
video_path = 'videos/test_policy.mp4'
# make a local copy
imageio.mimsave(video_path, frames, fps=30)
show_video('videos/test_policy.mp4')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107


In [18]:
print(f"Neural Data: {NeuralData[0][1]} \n length: {len(NeuralData[0][1])}") # 512 Neurons
print(f"Muscle Actuator Data: {MuscleActData[0][1]}\n length: {len(MuscleActData[0][1])}") # 63 Muscles
print(f"Fiber/Joint Angle/Joint Velocity Data: {FiberJointData[0][1]} \n length: {len(FiberJointData[0][1])}") # 63 muscles + 44 joints

Neural Data: [0.4829 0.4449 0.4658 0.4702 0.4433 0.4674 0.5478 0.517  0.4635 0.4603
 0.5074 0.5171 0.4813 0.4795 0.4977 0.4762 0.5161 0.4823 0.467  0.4911
 0.4668 0.5115 0.4805 0.4691 0.4502 0.493  0.4789 0.4913 0.4893 0.4537
 0.4894 0.5112 0.4841 0.4976 0.5349 0.5477 0.5312 0.4823 0.4491 0.5089
 0.4867 0.5156 0.5459 0.4996 0.4878 0.494  0.4997 0.4793 0.5115 0.4549
 0.4945 0.5054 0.5333 0.5107 0.5172 0.5298 0.5461 0.4709 0.4764 0.4482
 0.4352 0.4879 0.495  0.5518 0.5172 0.4539 0.4907 0.4597 0.5127 0.4914
 0.4681 0.4771 0.5527 0.4959 0.4762 0.4838 0.4387 0.5486 0.4983 0.4941
 0.4702 0.5518 0.4721 0.463  0.511  0.5114 0.4652 0.5646 0.4848 0.541
 0.4806 0.5136 0.5177 0.4565 0.4805 0.4824 0.5097 0.4721 0.5278 0.4942
 0.4557 0.5057 0.4757 0.5084 0.5174 0.5489 0.4817 0.4987 0.4686 0.507
 0.4926 0.4353 0.534  0.539  0.5052 0.456  0.4877 0.4971 0.495  0.4887
 0.5168 0.5353 0.4552 0.5484 0.5164 0.4756 0.4863 0.4375 0.4722 0.526
 0.4642 0.5313 0.4967 0.5062 0.521  0.5046 0.5525 0.497  0.5133 0.5

<p> We'll basically have 3 matrices A (number of rows = number of steps): <br>
    - Neurons: 512 columns for each neuron <br>
    - Muscle Activity: 63 columns for each muscle actuator intensity <br>
    - Fiber Length/Joint Angle/Velocity: 151 columns for combination of 63 fiber length, 44 joint angle and 44 joint velocities <br>

Create Matrix

In [21]:
import numpy as np
import scipy.io

time_steps = 16

# Create the `times` array and ensure it's double precision
times =  np.arange(0, time_steps*0.05, 0.05).reshape(-1, 1)
    
data = [NeuralData, MuscleActData, FiberJointData]
matrices = []

for x in range(3): # 3 Matrices
    data_struct = np.empty((ep_num,), dtype=[('A', 'O'), ('times', 'O')])

    for n in range(ep_num):
        data_struct['A'][n] = data[x][n] 
        data_struct['times'][n] = times  

    
    matrices.append(data_struct)
    
# Save the structured data to a .mat file with 'Data' as the matrix name
scipy.io.savemat('Neural_Data_wo_training.mat', {'Data': matrices[0]})
scipy.io.savemat('MuscleAct_Data_wo_training.mat', {'Data': matrices[1]})
scipy.io.savemat('FiberJoint_Data_wo_training.mat', {'Data': matrices[2]})
